# CSE 291 Assignment 2 BiLSTM CRF

## Download Data/Eval Script

In [ ]:
!wget https://raw.githubusercontent.com/sighsmile/conlleval/master/conlleval.py
!wget https://raw.githubusercontent.com/tberg12/cse291spr21/main/assignment2/train.data.quad
!wget https://raw.githubusercontent.com/tberg12/cse291spr21/main/assignment2/dev.data.quad

In [ ]:
import conlleval
from tqdm import tqdm
import numpy as np
from collections import defaultdict, Counter
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim
from torchtext.vocab import vocab
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

torch.manual_seed(291)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

## Data Preparation

In [ ]:
TRAIN_DATA = 'train.data.quad'
VALID_DATA = 'dev.data.quad'
UNK = '<unk>'
PAD = '<pad>'
START_TAG = "<start>"  # you can add this explicitly or use it implicitly in your CRF layer
STOP_TAG = "<stop>"    # you can add this explicitly or use it implicitly in your CRF layer


def read_conll_sentence(path):
    """ Read a CONLL-format sentence into vocab objects
    Args:
        :param path: path to CONLL-format data file
        :param word_vocab: Vocabulary object for source
        :param label_vocab: Vocabulary object for target
    """
    sent = [[], []]
    with open(path) as f:
        for line in f:
            line = line.strip().split()
            if line:
                # replace numbers with 0000
                word = line[0]
                word = '0000' if word.isnumeric() else word
                sent[0].append(word)
                sent[1].append(line[3])
            else:
                yield sent[0], sent[1]
                sent = [[], []]


def prepare_dataset(dataset, word_vocab, label_vocab):
    dataset = [
      [
        torch.tensor([word_vocab.get_stoi().get(word,0) for word in sent[0]], dtype=torch.long),
        torch.tensor([label_vocab.get_stoi()[label] for label in sent[1]], dtype=torch.long),
      ]
      for sent in dataset
    ]
    return dataset


# load a list of sentences, where each word in the list is a tuple containing the word and the label
train_data = list(read_conll_sentence(TRAIN_DATA))
train_word_counter = Counter([word for sent in train_data for word in sent[0]])
train_label_counter = Counter([label for sent in train_data for label in sent[1]])
word_vocab = vocab(train_word_counter, specials=(UNK, PAD), min_freq=2)
# print(word_vocab.get_stoi())
# word_vocab.set_default_index(UNK)
# label_vocab = vocab(train_label_counter, specials=(), min_freq=1)
label_vocab = vocab(train_label_counter, specials=(START_TAG,STOP_TAG), min_freq=1)
train_data = prepare_dataset(train_data, word_vocab, label_vocab)
print('Train word vocab:', len(word_vocab), 'symbols.')
print('Train label vocab:', len(label_vocab), f'symbols: {list(label_vocab.get_stoi().keys())}')
valid_data = list(read_conll_sentence(VALID_DATA))
valid_data = prepare_dataset(valid_data, word_vocab, label_vocab)
print('Train data:', len(train_data), 'sentences.')
print('Valid data:', len(valid_data))

print(' '.join([word_vocab.get_itos()[i.item()] for i in train_data[0][0]]))
print(' '.join([label_vocab.get_itos()[i.item()] for i in train_data[0][1]]))

print(' '.join([word_vocab.get_itos()[i.item()] for i in valid_data[1][0]]))
print(' '.join([label_vocab.get_itos()[i.item()] for i in valid_data[1][1]]))

## BiLSTMTagger

In [ ]:
# Starter code implementing a BiLSTM Tagger
# which makes locally normalized, independent
# tag classifications at each time step

class BiLSTMTagger(nn.Module):
    def __init__(self, vocab_size, tag_vocab_size, embedding_dim, hidden_dim, dropout=0.3):
        super(BiLSTMTagger, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tagset_size = tag_vocab_size
        self.word_embeds = nn.Embedding(vocab_size, embedding_dim).to(device)
        self.bilstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=1, bidirectional=True, batch_first=True).to(device)
        self.tag_projection_layer = nn.Linear(hidden_dim, self.tagset_size).to(device)
        self.dropout = nn.Dropout(p=dropout)

    def init_hidden(self):
        return (torch.randn(2, 1, self.hidden_dim // 2).to(device),
                torch.randn(2, 1, self.hidden_dim // 2).to(device))

    def compute_lstm_emission_features(self, sentence):
        hidden = self.init_hidden()
        embeds = self.dropout(self.word_embeds(sentence))
        bilstm_out, hidden = self.bilstm(embeds, hidden)
        bilstm_out = self.dropout(bilstm_out)
        bilstm_out = bilstm_out
        bilstm_feats = self.tag_projection_layer(bilstm_out)
        return bilstm_feats

    def forward(self, sentence):
        bilstm_feats = self.compute_lstm_emission_features(sentence)
        return bilstm_feats.max(-1)[0].sum(), bilstm_feats.argmax(-1)

    def loss(self, sentence, tags):
        bilstm_feats = self.compute_lstm_emission_features(sentence)
        # transform predictions to (n_examples, n_classes) and ground truth to (n_examples)
        return torch.nn.functional.cross_entropy(
              bilstm_feats.view(-1, self.tagset_size), 
              tags.view(-1), 
              reduction='sum'
            )


## Train / Eval loop

In [ ]:
def train(model, train_data, valid_data, word_vocab, label_vocab, epochs, log_interval=25):
    losses_per_epoch = []
    for epoch in range(epochs):
        print(f'--- EPOCH {epoch} ---')
        model.train()
        losses_per_epoch.append([])
        for i, (sent, tags) in enumerate(train_data):
            model.zero_grad()
            sent, tags = sent.to(device), tags.to(device)
            sent = sent.unsqueeze(0)
            tags = tags.unsqueeze(0)
            loss = model.loss(sent, tags)
            loss.backward()
            optimizer.step()

            losses_per_epoch[-1].append(loss.detach().cpu().item())
            if i > 0 and i % log_interval == 0:
                print(f'Avg loss over last {log_interval} updates: {np.mean(losses_per_epoch[-1][-log_interval:])}')

        evaluate(model, valid_data, word_vocab, label_vocab,epoch)
loss_list=[]
best_loss=float('inf')
def evaluate(model, dataset, word_vocab, label_vocab,epoch):
    model.eval()
    global best_loss 
    losses = []
    scores = []
    true_tags = []
    pred_tags = []
    sents = []
    for i, (sent, tags) in enumerate(dataset):
        with torch.no_grad():
            sent, tags = sent.to(device), tags.to(device)
            sent = sent.unsqueeze(0)
            tags = tags.unsqueeze(0)
            losses.append(model.loss(sent, tags).cpu().detach().item())
            score, pred_tag_seq = model(sent)
            scores.append(score.cpu().detach().numpy())
            true_tags.append([label_vocab.get_itos()[i] for i in tags.tolist()[0]])
            pred_tags.append([label_vocab.get_itos()[i] for i in pred_tag_seq[0]])
            sents.append([word_vocab.get_itos()[i] for i in sent[0]])

    print('Avg evaluation loss:', np.mean(losses))
    loss_list.append(np.mean(losses))
    if np.mean(losses)<best_loss: 
        best_loss=np.mean(losses)
        print('best_loss',best_loss)
        torch.save(model, 'baseline/best-model{}.pt'.format(epoch))
        torch.save(model.state_dict(), 'baselineparam/best-model-param{}.pt'.format(epoch))
    print(conlleval.evaluate([tag for tags in true_tags for tag in tags], [tag for tags in pred_tags for tag in tags], verbose=True))
    print('\n5 random evaluation samples:')
    for i in np.random.randint(0, len(sents), size=5):
        print('SENT:', ' '.join(sents[i]))
        print('TRUE:', ' '.join(true_tags[i]))
        print('PRED:', ' '.join(pred_tags[i]))
    return sents, true_tags, pred_tags


## Training

In [ ]:
# Train BiLSTM Tagger Baseline
model = BiLSTMTagger(len(word_vocab), len(label_vocab), 128, 256).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
train(model, train_data, valid_data, word_vocab, label_vocab, epochs=12, log_interval=500)

## CRF

In [ ]:
def argmax(vec):
    _, idx = torch.max(vec, 1)
    return idx.item()

def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))


class BiLSTMCRFTagger(nn.Module):
    def __init__(self, vocab_size, tag_vocab, embedding_dim, hidden_dim, dropout=0.3):
        super(BiLSTMCRFTagger, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_vocab = tag_vocab
        self.tagset_size = len(tag_vocab)
        self.word_embeds = nn.Embedding(vocab_size, embedding_dim).to(device)
        self.bilstm = nn.LSTM(embedding_dim, hidden_dim // 2, num_layers=1, bidirectional=True).to(device)
        self.tag_projection_layer = nn.Linear(hidden_dim, self.tagset_size).to(device)
        self.dropout = nn.Dropout(p=dropout)
        self.transitions = nn.Parameter(torch.randn(self.tagset_size, self.tagset_size))
        self.transitions.data[tag_vocab[START_TAG], :] = -10000
        self.transitions.data[:, tag_vocab[STOP_TAG]] = -10000
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)



    def init_hidden(self):
        return (torch.randn(2, 1, self.hidden_dim // 2).to(device),
                torch.randn(2, 1, self.hidden_dim // 2).to(device))

    def compute_lstm_emission_features(self, sentence):
        self.hidden = self.init_hidden()
        embeds = self.dropout(self.word_embeds(sentence).view(len(sentence), 1, -1))
        lstm_out, self.hidden = self.bilstm(embeds, self.hidden)
        lstm_out = self.dropout(lstm_out)
        lstm_out = lstm_out.view(len(sentence), self.hidden_dim)
        lstm_feats = self.hidden2tag(lstm_out)
        return lstm_feats
    
    def score_sent(self, feats, tags):
        score = torch.zeros(1)
        tags = torch.cat([torch.tensor([self.tag_vocab[START_TAG]], dtype=torch.long), tags])
        for i, feat in enumerate(feats):
            score = score + \
                self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_vocab[STOP_TAG], tags[-1]]
        return score
    
    def forward_p_algo(self, feats):
        alphas = torch.full((1, self.tagset_size), -10000.)
        alphas[0][self.tag_vocab[START_TAG]] = 0.
        forward_var = alphas
        for feat in feats:
            timestep_alphas = [] 
            for t_next in range(self.tagset_size):
                score_e = feat[t_next].view(1, -1).expand(1, self.tagset_size)
                score_t = self.transitions[t_next].view(1, -1)
                next_tag = forward_var + score_t + score_e
                timestep_alphas.append(log_sum_exp(next_tag).view(1))
            forward_var = torch.cat(timestep_alphas).view(1, -1)
        var_terminal = forward_var + self.transitions[self.tag_vocab[STOP_TAG]]
        alpha = log_sum_exp(var_terminal)
        return alpha
    
    def decode(self, feats):
        bptr = []
        ver_vars = torch.full((1, self.tagset_size), -10000.)
        ver_vars[0][self.tag_vocab[START_TAG]] = 0
        forward_var = ver_vars
        for feat in feats:
            bptr_t = [] 
            ver_vars_t = [] 
            for t_next in range(self.tagset_size):
                next_tag = forward_var + self.transitions[t_next]
                best_tag= argmax(next_tag)
                bptr_t.append(best_tag)
                ver_vars_t.append(next_tag[0][best_tag].view(1))
            forward_var = (torch.cat(ver_vars_t) + feat).view(1, -1)
            bptr.append(bptr_t)

        var_terminal = forward_var + self.transitions[self.tag_vocab[STOP_TAG]]
        best_tag = argmax(var_terminal)
        path_score = var_terminal[0][best_tag]
        best_path = [best_tag]
        for bptr_t in reversed(bptr):
            best_tag = bptr_t[best_tag]
            best_path.append(best_tag)
        start = best_path.pop()
        assert start == self.tag_vocab[START_TAG] 
        best_path.reverse()
        return path_score, best_path
    
    def forward(self, sentence): 
        lstm_feats = self.compute_lstm_emission_features(sentence)
        score, tag_seq = self.decode(lstm_feats)
        return score, tag_seq
        
    def loss(self, sentence, tags):
        feats = self.compute_lstm_emission_features(sentence)
        num_score = self.score_sent(feats, tags)
        partition_score = self.forward_p_algo(feats)
        return partition_score - num_score


In [ ]:
def train_crf(model, train_data, valid_data, word_vocab, label_vocab, epochs, log_interval=25):
    losses_per_epoch = []
    for epoch in range(epochs):
        print(f'--- EPOCH {epoch} ---')
        model.train()
        losses_per_epoch.append([])
        for i, (sent, tags) in enumerate(train_data):
            model.zero_grad()
            sent, tags = sent.to(device), tags.to(device)
            # sent = sent.unsqueeze(0)
            # tags = tags.unsqueeze(0)
            loss = model.loss(sent, tags)
            loss.backward()
            optimizer.step()

            losses_per_epoch[-1].append(loss.detach().cpu().item())
            if i > 0 and i % log_interval == 0:
                print(f'Avg loss over last {log_interval} updates: {np.mean(losses_per_epoch[-1][-log_interval:])}')

        evaluate_crf(model, valid_data, word_vocab, label_vocab,epoch)

loss_list_crf=[]    
best_loss_crf=float('inf')
def evaluate_crf(model, dataset, word_vocab, label_vocab,epoch):
    model.eval()
    global best_loss_crf 
    losses = []
    scores = []
    true_tags = []
    pred_tags = []
    sents = []
    for i, (sent, tags) in enumerate(dataset):
        with torch.no_grad():
            sent, tags = sent.to(device), tags.to(device)
            # sent = sent.unsqueeze(0)
            # tags = tags.unsqueeze(0)
            losses.append(model.loss(sent, tags).cpu().detach().item())
            score, pred_tag_seq = model(sent)
            scores.append(score.cpu().detach().numpy())
            true_tags.append([label_vocab.get_itos()[i] for i in tags.tolist()])
            pred_tags.append([label_vocab.get_itos()[i] for i in pred_tag_seq])
            sents.append([word_vocab.get_itos()[i] for i in sent])

    print('Avg evaluation loss:', np.mean(losses))
    loss_list_crf.append(np.mean(losses))
    if np.mean(losses)<best_loss_crf: 
        best_loss_crf=np.mean(losses)
        print(best_loss_crf)
        torch.save(model, 'baselinecrf/best-model{}.pt'.format(epoch))
        torch.save(model.state_dict(), 'baselineparamcrf/best-model-param{}.pt'.format(epoch))
    print(conlleval.evaluate([tag for tags in true_tags for tag in tags], [tag for tags in pred_tags for tag in tags], verbose=True))
    print('\n5 random evaluation samples:')
    for i in np.random.randint(0, len(sents), size=5):
        print('SENT:', ' '.join(sents[i]))
        print('TRUE:', ' '.join(true_tags[i]))
        print('PRED:', ' '.join(pred_tags[i]))
    return sents, true_tags, pred_tags

model = BiLSTMCRFTagger(len(word_vocab), label_vocab, 128, 256).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
train_crf(model, train_data, valid_data, word_vocab, label_vocab, epochs=12, log_interval=500)

In [ ]:
loss_list

In [ ]:
loss_list_crf

In [ ]:
epochlis=[i for i in range(1,len(loss_list_crf)+1)]

In [ ]:
import matplotlib.pyplot as plt
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.scatter(epochlis,loss_list)
plt.scatter(epochlis,loss_list_crf)
plt.plot(epochlis,loss_list, label='Baseline BiLSTm')
plt.plot(epochlis,loss_list_crf, label='CRF BiLSTM')
plt.legend()
plt.show()

In [ ]:
model.load_state_dict(torch.load('baselineparam/best-model-param4.pt'))
evaluate(model, valid_data, word_vocab, label_vocab,5)

In [ ]:
model.load_state_dict(torch.load('baselineparamcrf/best-model-param5.pt'))
evaluate_crf(model, valid_data, word_vocab, label_vocab,6)